In [82]:
import networkx as nx
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import sys
import math
import pickle
import jsonpickle

In [111]:
n_tiles = pd.read_csv('../qgis/hexagons_centers.csv').shape[0]
n_tiles

1590

In [2]:
stops_with_ids = pd.read_csv("../qgis/stops_with_tile_ids.csv", usecols = ['ID','stop_id'])
stops_with_ids = stops_with_ids.set_index('stop_id')
print(stops_with_ids.shape)
stops_with_ids.head()

(24499, 1)


,ID
stop_id,
132,553
133,949
134,892
135,603
136,572


# Computing centers of hexagons

## Graph 
For complete set of bus and train stops

In [3]:
G = nx.read_gpickle('../graph/complete.graph')

In [4]:
stops_with_ids['degree'] = 0

For the hexagons where no affluence data is available from official **SBB/CFF** sources, we use the max node-degree in the cell as affluence.

### Strategy - choose node with min-average time in tile
For each tile get node with minimum average time to other nodes. 

`for tile:
    min_time = inf.
    matrix = distance(affluence_nodes + nodes ` $\in$ ` group)
    min_time = min(avg_time(matrix), min_time)`

Get all nodes belonging to both known stop_ids in grid and the directed graph.

In [93]:
gb = stops_with_ids.groupby('ID')
affluence_ids = affluence.stop_id
for k, gp in gb:
    matrix = 

2
3
4
5
6
7
9
10
11
12
13
14
15
16
20
21
22
23
24
25
26
28
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
49
50
51
52
53
54
57
58
59
60
61
62
64
65
68
69
70
71
72
73
74
75
76
79
80
81
82
83
84
85
86
87
88
90
91
92
93
94
95
96
97
98
99
100
102
103
104
105
106
107
108
109
110
111
112
113
114
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
137
138
139
140
141
143
144
145
146
147
148
149
150
152
153
154
155
156
157
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
182
185
186
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
210
211
218
219
220
221
222
223
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
247
248
249
250
251
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
279
281
282
283
284
285
287
288
289
290
291
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
314
315
316
317
318
319
320
322
32

In [35]:
nodes = G.nodes()
full_nodes = []
known_stops = stops_with_ids.index
for n in nodes:
    if n in known_stops:
        full_nodes.append(n)

In [69]:
with open('../res/full_nodes', 'wb') as f:
    pickle.dump(full_nodes, f)

Brouillon

In [63]:
with open('../res/matrix', 'r') as n_file:
    matrix = jsonpickle.decode(n_file.read())

With `full_nodes`, use `process_graph` script to generate full graph of time-distances between nodes. 

Use result to compute average time-distance for each node, then groupby tile IDs and pick the minimum for each tile.

In [ ]:
n_to_i = {}
i = 0
for n in full_nodes:
    n_to_i[n] = i

In [61]:
with open('../res/full_matrix', 'r') as matrix_file:
    full_matrix = jsonpickle.decode(matrix_file.read())
    
with open('../res/full_nodes', 'r') as nodes_file:
    full_nodes = jsonpickle.decode(nodes_file.read())

FileNotFoundError: [Errno 2] No such file or directory: '../res/matrix_small'

### 2nd strategy - max degree of node in tile

In [68]:
nodes = G.nodes()

errors = 0
for n in nodes:
    try:
        stops_with_ids.loc[n].degree = G.degree(n) 
    except KeyError:
        errors += 1
        
print("Nodes missing in graph: {}/{}, when matched with `stops_with_ids`".format(errors, len(nodes)))

Nodes missing in graph: 276/22056, when matched with `stops_with_ids`


### NEXT

Groupby the hexagon ID and aggregate by max affluence.

In [36]:
grouped_by_degree = stops_with_ids.reset_index().groupby('ID')['stop_id', 'degree'].agg({'degree': max})
grouped_by_degree.columns = grouped_by_degree.columns.droplevel()
# grouped_by_degree = grouped_by_degree[grouped_by_degree.degree != 0]
grouped_by_degree.head()

,stop_id,degree
ID,,
2,8595012,0
3,8595953,0
4,8595951,0
5,8595553,0
6,8595952,0


In [37]:
grouped_by_degree[grouped_by_degree.degree != 0].shape[0]

0

In [38]:
grouped_by_degree[grouped_by_degree.stop_id == 8530043]

,stop_id,degree
ID,,
255,8530043,0


In [39]:
missing = grouped_by_degree[grouped_by_degree.degree == 0].shape[0]
total = grouped_by_degree.shape[0]
print("After grouping, missing affluence data for {}/{} cells".format(missing, total))

After grouping, missing affluence data for 1259/1259 cells


## Affluence for available nodes

In [40]:
affluence = pd.read_csv("../gtfs/passagierfrequenz.csv", sep=';')
affluence['x'], affluence['y'] = affluence.geopos.str.split(',', 1).str
affluence = affluence.drop(['Bahnhof_Haltestelle', 'DWV', 'Bemerkungen', 'lod', 'geopos', 'Eigner', 'Bezugsjahr'], axis=1)
affluence.columns = ['Code', 'Affluence', 'x', 'y']
affluence['stop_id'] = 0

In [41]:
affluence.head()

,Code,Affluence,x,y,stop_id
0,AAT,770,47.3359563788,8.76561022548,0
1,AE,2100,47.4677359724,7.60305476436,0
2,ALL,3000,46.4757395273,6.39970400408,0
3,AW,3000,47.5504471564,9.30221759023,0
4,ARN,210,47.4420020412,9.25200601938,0


In [42]:
affluence.to_csv('../gtfs/affluence_code.csv')

In [83]:
affluence_with_ids = pd.read_csv("../qgis/affluence_with_tile_ids.csv")

In [44]:
joined_aff = affluence.join(affluence_with_ids, rsuffix='_r').drop(
        ['x', 'y', 'Code_r', 'stop_id', 'Code'], axis=1)
grouped_by_affluence = joined_aff.groupby('ID').agg(max).reset_index()

print("Centers with affluence data: {}".format(grouped_by_affluence.shape[0]))

Centers with affluence data: 371


## Combine results
Is there relation between degree of node and affluence?


In [45]:
affluence = affluence.join(affluence_with_ids, rsuffix='_r').drop(['Code_r'], axis=1)
affluence.head()

,Code,Affluence,x,y,stop_id,ID
0,AAT,770,47.3359563788,8.76561022548,0,1039.0
1,AE,2100,47.4677359724,7.60305476436,0,525.0
2,ALL,3000,46.4757395273,6.39970400408,0,50.0
3,AW,3000,47.5504471564,9.30221759023,0,1337.0
4,ARN,210,47.4420020412,9.25200601938,0,1309.0


In [46]:
stops = pd.read_csv('../gtfs/stops.txt').drop(['Unnamed: 0', 'platform_code'], axis=1)

In [47]:
stops.head()

,stop_id,stop_lon,stop_lat
0,132,7.689360,47.196374
1,133,8.603653,46.154371
2,134,8.435913,46.538322
3,135,7.773846,46.356888
4,136,7.717215,46.433756


### Get closest stop match by euclidean distance between coordinates -> heuristic 
Result stored in `gtfs/affluence_with_stopid.csv`

**WARNING**: takes some time to compute

In [ ]:
MAX_SIZE = sys.maxsize
for i in range(affluence.shape[0]):
    min_ = MAX_SIZE
    min_id = None
    x1 = float(affluence.loc[i].x)
    y1 = float(affluence.loc[i].y)
    for j in range(stops.shape[0]):
        x2 = float(stops.loc[j].stop_lat)
        y2 = float(stops.loc[j].stop_lon)
        dist = math.sqrt(pow(abs(x1-x2),2) + pow(abs(y1 - y2),2))
        if dist < min_:
            min_ = dist
            min_id = stops.loc[j].stop_id
    affluence.set_value(i, 'stop_id', min_id)
    if (i%25 == 0):
        print("{}/{}".format(i, affluence.shape[0]))
        
affluence.to_csv('gtfs/affluence_with_stopid.csv')    

**Instead** use pre-computed file below

In [49]:
affluence = pd.read_csv('../gtfs/affluence_with_stopid.csv')
affluence.drop(['Unnamed: 0'], axis=1, inplace=True)
affluence.head()

,Code,Affluence,x,y,stop_id,ID
0,AAT,770,47.335956,8.765610,8503124,1039.0
1,AE,2100,47.467736,7.603055,8500117,525.0
2,ALL,3000,46.475740,6.399704,8501035,50.0
3,AW,3000,47.550447,9.302218,8506109,1337.0
4,ARN,210,47.442002,9.252006,8506211,1309.0


Merge max affluence nodes for each cell with corresponding stop_id (gtfs format).

In [50]:
affluence_stop_ids = grouped_by_affluence.merge(affluence, left_on=['ID', 'Affluence'],
                          right_on=['ID', 'Affluence'])[['ID', 'stop_id']]

In [51]:
affluence_stop_ids.head()

,ID,stop_id
0,3.0,8501001
1,6.0,8501003
2,11.0,8516155
3,12.0,8501008
4,22.0,8501022


Combine the results from affluence and degree to one collection

In [52]:
cell_centers = grouped_by_degree.drop(['degree'], axis=1)
for i in range(affluence_stop_ids.shape[0]):
    cell_id = affluence_stop_ids.loc[i].ID
    stop_id = affluence_stop_ids.loc[i].stop_id
    cell_centers.set_value(cell_id, 'stop_id', stop_id)
    

Get list of all centers

## IMPORTANT: sort the centers by tile-id

In [118]:
cell_centers = cell_centers.reindex(range(1, n_tiles+1), fill_value=0)

In [119]:
lst_centers = list(map(lambda x: int(x), cell_centers.stop_id))

In [126]:
nodes = G.nodes()
filt_centers = []
for c in lst_centers:
    if c in nodes:
        filt_centers.append(c)
    else: 
        filt_centers.append(0)

In [129]:
with open('../res/center_nodes', 'wb') as f:
    pickle.dump(filt_centers, f)